In [8]:
using XLSX;
using CSV;
using DataFrames;
using Plots;
using Dates;
using Statistics;
using StatsPlots
import PyPlot;
using LinearAlgebra

In [9]:
pwd()

"/home/lqxc/Lab/data-process/lab1/julia"

In [31]:
quality =
"file/xian_guangdian.csv" |>
CSV.File |>
DataFrame |>
data ->
    begin
        combine(nrow, groupby(select(data, :客户等级), :客户等级)) |>
        data -> rename(data, :nrow => "用户数量") |> println
        combine(nrow, groupby(select(data, [:客户等级, :网络类型]), [:客户等级, :网络类型]))
    end |>
    data ->
        rename(data, :nrow => :quantity, :网络类型 => :net_kind, :客户等级 => :user_level)
data = combine(groupby(quality, :net_kind), [:user_level, :quantity])
dict = Dict(
"5星ABD客户" => "star_5ABD",
"离线" => "out_link",
"3星AB客户" => "star_3AB",
"1星D客户" => "star_1D",
"1星A客户" => "star_1A",
"VIP商业个人客户" => "vip",
"3星AD客户" => "start_3AD",
)
1:(data|>nrow) .|>
i -> begin
data[i, :net_kind] =
    Dict("农网用户" => "village", "城网用户" => "city", " " => "unknown")[data[
        i,
        :net_kind,
    ]]
data[i, :user_level] = dict[data[i, :user_level]]
end
gp = groupby(data, :net_kind)
gp |>
keys .|>
kind -> @df combine(gp[kind], [:user_level, :quantity]) plot(
:user_level,
:quantity,
label = "$kind",
) |> fig -> savefig(fig, "images/first_$kind");

7×2 DataFrame
 Row │ 客户等级         用户数量 
     │ String31         Int64    
─────┼───────────────────────────
   1 │ 5星ABD客户           3695
   2 │ 离线                  498
   3 │ 3星AB客户             151
   4 │ 1星D客户              403
   5 │ 3星AD客户              76
   6 │ VIP商业个人客户        63
   7 │ 1星A客户              113


In [29]:
file_path = "file/xian_beijing_salary.xlsx"
salarys = DataFrame(XLSX.readdata(file_path, "Sheet1!C3:D14"), :auto) .|> identity
salary = [salarys.x1, salarys.x2]

println("mean:$(salary .|> Statistics.mean)")
println("var:$(salary .|> Statistics.var)")
println("std:$(salary .|> Statistics.std)")
println("cov:$(salary |> Statistics.cov)")
println("cor:$(salary .|> Statistics.cor)")

violin(["Xi'an"], salarys.x1, label = "Xi'an")
violin!(["Beijing"], salarys.x2, label = "Beijing") |>
fig -> savefig(fig, "images/violin")
boxplot(["Xi'an"], salarys.x1, label = "Xi'an")
boxplot!(["Beijing"], salarys.x2, label = "Beijing") |>
fig -> savefig(fig, "images/box")

mean:[77.58333333333333, 97.83333333333333]
var:[435.71969696969694, 492.8787878787878]
std:[20.8738998984305, 22.20087358368557]
cov:[8.0 -40.0 -46.0 -54.0 -52.0 -54.0 -46.0 -40.0 -52.0 -50.0 -60.0 0.0; -40.0 200.0 230.0 270.0 260.0 270.0 230.0 200.0 260.0 250.0 300.0 0.0; -46.0 230.0 264.5 310.5 299.0 310.5 264.5 230.0 299.0 287.5 345.0 0.0; -54.0 270.0 310.5 364.5 351.0 364.5 310.5 270.0 351.0 337.5 405.0 0.0; -52.0 260.0 299.0 351.0 338.0 351.0 299.0 260.0 338.0 325.0 390.0 0.0; -54.0 270.0 310.5 364.5 351.0 364.5 310.5 270.0 351.0 337.5 405.0 0.0; -46.0 230.0 264.5 310.5 299.0 310.5 264.5 230.0 299.0 287.5 345.0 0.0; -40.0 200.0 230.0 270.0 260.0 270.0 230.0 200.0 260.0 250.0 300.0 0.0; -52.0 260.0 299.0 351.0 338.0 351.0 299.0 260.0 338.0 325.0 390.0 0.0; -50.0 250.0 287.5 337.5 325.0 337.5 287.5 250.0 325.0 312.5 375.0 0.0; -60.0 300.0 345.0 405.0 390.0 405.0 345.0 300.0 390.0 375.0 450.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
cor:[1, 1]


In [28]:
file_path = "file/3movie_metadata.csv"
movie_metadata = file_path |> CSV.File |> DataFrame
dropmissing!(movie_metadata, :director_name)
dict = Dict(
    :color => "Color",
    :num_critic_for_reviews => 0,
    :duration => 0,
    :director_facebook_likes => 0,
    :actor_3_facebook_likes => 0,
    :actor_2_name => "",
    :actor_1_facebook_likes => 0,
    :gross => 0,
    :genres => "",
    :actor_1_name => "",
    :movie_title => "",
    :num_voted_users => 0,
    :cast_total_facebook_likes => 0,
    :actor_3_name => "",
    :facenumber_in_poster => 0,
    :plot_keywords => "",
    :movie_imdb_link => "",
    :num_user_for_reviews => 0,
    :language => "",
    :country => "",
    :content_rating => 0,
    :budget => 0,
    :title_year => 0,
    :actor_2_facebook_likes => 0,
    :imdb_score => 0,
    :aspect_ratio => 0,
    :movie_facebook_likes => 0,
)
dict |>
keys .|>
key -> transform!(
    movie_metadata,
    :,
    key => (col -> col .|> each -> if ismissing(each)
        dict[key]
    else
        each
    end) => key,
)

27-element Vector{DataFrame}:
 4939×28 DataFrame
  Row │ color      director_name       num_critic_for_reviews  duration  direc ⋯
      │ Abstract…  String63            Int64                   Int64     Int64 ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ Color      James Cameron                          723       178        ⋯
    2 │ Color      Gore Verbinski                         302       169
    3 │ Color      Sam Mendes                             602       148
    4 │ Color      Christopher Nolan                      813       164
    5 │ Color      Doug Walker                              0         0        ⋯
    6 │ Color      Andrew Stanton                         462       132
    7 │ Color      Sam Raimi                              392       156
    8 │ Color      Nathan Greno                           324       100
  ⋮   │     ⋮              ⋮                     ⋮                ⋮            ⋱
 4933 │ Color      Robert Rodrigu

In [18]:
["file/4ReaderInformation.csv", "file/4ReaderRentRecode.csv"] .|>
CSV.File .|>
DataFrame |>
dates -> begin
    println(dates...)
    innerjoin(dates..., on = :num) |>
    file -> begin
        file |> println
        CSV.write("file/join.csv", file)
    end
end

10×4 DataFrame
 Row │ num    sex      institution  category 
     │ Int64  String1  String7      String7  
─────┼───────────────────────────────────────
   1 │     1  m        xupt         teacher
   2 │     2  w        xupt         teacher
   3 │     3  m        xupt         student
   4 │     4  w        xupt         student
   5 │     5  m        xupt         student
   6 │     6  m        xupt         student
   7 │     7  w        xupt         student
   8 │     8  w        xupt         teacher
   9 │     9  m        xupt         teacher
  10 │    10  w        xupt         teacher10×4 DataFrame
 Row │ num    name     book                               date    
     │ Int64  String7  String63                           Float64 
─────┼────────────────────────────────────────────────────────────
   1 │     1  Tom      gone with the wind                    3.1
   2 │     2  Anna     The scarlet letter                    4.3
   3 │     3  Jerry    The adventures of Tom Sawyer          3


10×7 DataFrame
 Row │ num    sex      institution  category  name     book                               date    
     │ Int64  String1  String7      String7   String7  String63                           Float64 
─────┼────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1  m        xupt         teacher   Tom      gone with the wind                    3.1
   2 │     2  w        xupt         teacher   Anna     The scarlet letter                    4.3
   3 │     3  m        xupt         student   Jerry    The adventures of Tom Sawyer          3.18
   4 │     4  w        xupt         student   Cyning   Tales of two cities                   5.21
   5 │     5  m        xupt         student   Peter    Pride and Prejudice                   5.3
   6 │     6  m        xupt         student   Kevien   Uncle Tom's Cabin                     4.27
   7 │     7  w        xupt         student   June     The old man and the sea               3.3
   8 

"file/join.csv"

In [16]:
mat = "file/5iris.csv" |> CSV.File |> DataFrame
    gp = groupby(mat, :Species)
    names = [:Sepal_length, :Sepal_width, :Petal_length, :Petal_width, :Species]
    @df gp[1] plot(
        :Sepal_length,
        :Sepal_width,
        :Petal_length,
        zcolor = reverse(:Petal_width),
        m = (10, 0.8, :blues, Plots.stroke(0)),
        fontfamily = "Yahei",
        xlabel = "Sepal_length",
        ylabel = "Sepal_width",
        zlabel = "Petal_length",
        title = "山鸢尾",
        label = "山鸢尾",
        w = 0,
    )
    @df gp[2] plot(
        :Sepal_length,
        :Sepal_width,
        :Petal_length,
        zcolor = reverse(:Petal_width),
        m = (10, 0.8, :blues, Plots.stroke(0)),
        fontfamily = "Yahei",
        xlabel = "Sepal_length",
        ylabel = "Sepal_width",
        zlabel = "Petal_length",
        title = "变色鸢尾",
        label = "变色鸢尾",
        w = 0,
    )
    @df gp[3] plot(
        :Sepal_length,
        :Sepal_width,
        :Petal_length,
        zcolor = reverse(:Petal_width),
        m = (10, 0.8, :blues, Plots.stroke(0)),
        fontfamily = "Yahei",
        xlabel = "Sepal_length",
        ylabel = "Sepal_width",
        zlabel = "Petal_length",
        title = "维吉尼亚鸢尾",
        label = "维吉尼亚鸢尾",
        w = 0,
    )
    data = copy(mat)
    data = select!(data, Not([:Species]))
    transform!(
        data,
        :,
        :Sepal_length => (x -> x .- mean(x)) => :Sepal_length,
        :Sepal_width => (x -> x .- mean(x)) =>:Sepal_width,
        :Petal_length => (x -> x .- mean(x)) =>:Petal_length,
        :Petal_width => (x -> x .- mean(x)) =>:Petal_width,
    )
    data = data |> Matrix
    (values, vectors) = data |> Statistics.cov |> eigen
    # sortperm 会将元素从小到大排序
    p = last(sortperm(values), 2) |> x -> vectors[:, x]
    data * p

150×2 Matrix{Float64}:
  0.326607    2.68421
 -0.169557    2.71539
 -0.137346    2.88982
 -0.311124    2.74644
  0.333925    2.72859
  0.747783    2.2799
 -0.0821045   2.82089
  0.170405    2.62648
 -0.570798    2.88796
 -0.106692    2.67384
  ⋮          
  0.409271   -1.92245
 -0.574925   -1.41407
  0.275975   -2.56332
  0.303504   -2.41939
  0.187415   -1.94402
 -0.375021   -1.52566
  0.0785192  -1.76405
  0.115877   -1.90163
 -0.282887   -1.38967